# Operation Safe Passage: Mission Controller

The Mission Controller is the central orchestrator for OSP.  
It connects:

- The map (`network.json`) generated in the Map Creation step.
- Agents (UAVs, UGVs) defined in `params.json`.
- The Environment, which manages nodes, terrain, and hazards.
- Time accounting for traversal, scanning, and mine-clearing.

The controller validates actions, updates simulation time, and returns the mission state.

In [ ]:
# Standard libraries
import os
import json

# Mission controller and agent classes
from operation_safe_passage.controller.mission_controller import MissionController


## Required Files

Two inputs are required:

1. **network.json**  
   - Generated by the Map Creation step.  
   - Contains nodes, edges, neighbors, start and end identifiers.

2. **params.json**  
   - Defines agents and scanner parameters.  
   - Must include:
     - `agents`: UAV list and UGV count
     - `scanner_params`: scanner definitions (referenced by UAVs)
     - `processing_params`: traversal and mine-clear timings
     - `terrain_coeff`: coefficients for terrain effects

In [ ]:
# Minimal configuration for testing MissionController (presaved in params_test.json)
params = {
    "processing_params": {
        "UGV_traversal_time": 20,
        "UGV_clear_time": 60,
        "UAV_traversal_time": 1
    },
    "scanner_params": {
  "scanners": {
    "default": {
      "visibility_scale": 0.55,
      "visibility_metric": 6,
      "kappa": 3.0,
      "noise_scale": 3.0,
      "noise_std": 0.75,
      "threshold": 0.5,
      "time": 1
    },
    "baseline": {
      "visibility_scale": 0.55,
      "visibility_metric": 6,
      "kappa": 3.0,
      "noise_scale": 3.0,
      "noise_std": 0.75,
      "threshold": 0.5,
      "time": 30
            }
        }
    },
    "agents": {
        "uavs": [
            { "name": "uav_alpha", "scanners": ["default"] }
        ],
        "num_ugvs": 1
    },
    "terrain_coeff": {"Grassy": 1.0}
}

In [ ]:
# Create MissionController using the config and map generated earlier
controller = MissionController(
    param_path="config/params_test.json",
    network_path="config/premade_network.json",  # must exist from Map Creation step
    output_dir="output"
)

# Inspect the list of initialized agents
[a.name for a in controller.agents]


['uav_alpha', 'UGV_0']

## Stepping Through the Mission

The controller advances with the `step()` method.

Actions are provided as a dictionary:
- Key: agent object
- Value: dict of requested `move` and (if UAV) `scan`

Moves:
- Direction index `0–5` or name (`"E"`, `"NE"`, …).
- `"noop"` or `None` for no movement.

Scans:
- Scanner name (UAV only).


In [ ]:
# Define actions: UAV scans, UGV moves
actions = {
    controller.agents[0]: {"move": "NE", "scan": "default"},  # UAV
    controller.agents[1]: {"move": "E"}                       # UGV
}

# Advance the mission by one step
state = controller.step(actions)

# Show updated time and agent states
print("Time:", state["time"])


## Mission State

The public mission state returned by `get_state()` contains:

- `time`: current simulation time.
- `agents`: list of agent dictionaries with:
  - `type`: agent name
  - `current`: metadata of current node (terrain, weather, etc.)
  - `neighbors`: metadata of adjacent nodes
  - `distance_to_goal`: hex distance to goal
  - `distance_to_goal_weighted`: weighted distance
  - `distance_to_agents`: hex distance to other agents
  - `previous_node`: last visited node
  - `previous_action`: last action taken
  - `num_moves`: count of moves taken

In [ ]:
# Fetch the current state explicitly
state = controller.get_state()

# Display top-level time and first agent summary
print("Time:", state["time"])
print(json.dumps(state["agents"][0], indent=2))


## Summary

- The Mission Controller connects configuration (`params.json`) and map (`network.json`).
- Agents are instantiated at the start node with traversal and scanning parameters.
- Each `step()` validates actions, applies time penalties, and updates the environment.
- `get_state()` provides the current public mission state for agents and metrics.

Next:
- See the **Agent Creation notebook** to define new UAV/UGV agents.
- Use the **Gym Environment notebook** to integrate the controller into reinforcement learning pipelines.
